In [1]:
import pandas as pd
import cv2
import numpy as np
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles
from utils import *

In [2]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),
                  'z{}'.format(val), 'v{}'.format(val)]

In [3]:
play_video_from_file('./videos/oneFoot.avi')

In [4]:
apply_mediapipe_holistic_model('./videos/oneFoot.avi')

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
